In [7]:
import pandas as pd
import pprint
from messari.messari import Messari
from tabulate import tabulate
from dotenv import load_dotenv
import os

In [8]:
dotenv_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path)
messari_api = os.getenv("MESSARI_API")

In [9]:
messari = Messari(messari_api)
assetsData = messari.get_all_assets(limit=500)["data"]

In [10]:
columns = list(pd.DataFrame(assetsData).columns)
metrics = list(assetsData[0]["metrics"].keys())
profile = list(assetsData[0]["profile"].keys())

metricsKeys = {}
profileKeys = {}

for i in range(len(metrics)):
    try:
        metricsKeys[metrics[i]] = list(assetsData[0]["metrics"][metrics[i]].keys())
    except:
        pass

for i in range(len(profile)):
    try:
        profileKeys[profile[i]] = list(assetsData[0]["profile"][profile[i]].keys())
    except:
        pass



print(columns)
print("\n")
print(metricsKeys.keys())
print("\n")
print(profileKeys.keys())
#print("\n")
#print(symbols)

['id', 'serial_id', 'symbol', 'name', 'slug', 'contract_addresses', '_internal_temp_agora_id', 'metrics', 'profile']


dict_keys(['market_data', 'marketcap', 'supply', 'blockchain_stats_24_hours', 'market_data_liquidity', 'all_time_high', 'cycle_low', 'token_sale_stats', 'mining_stats', 'developer_activity', 'roi_data', 'roi_by_year', 'risk_metrics', 'misc_data', 'reddit', 'on_chain_data', 'exchange_flows', 'miner_flows', 'supply_activity', 'supply_distribution'])


dict_keys(['general', 'contributors', 'advisors', 'investors', 'ecosystem', 'economics', 'technology', 'governance', 'metadata'])


In [11]:
masterlist = []

useMetricKeys = ["market_data", "marketcap", "supply", "token_sale_stats", "developer_activity", "roi_data", "roi_by_year", "risk_metrics", "supply_activity", "supply_distribution"]
useProfKeys = ['economics', 'governance']
# for each asset
for asset in range(len(assetsData)):
    assetdict = {}
    assetdict["name"] =  assetsData[asset]["name"]
    assetdict["symbol"] = assetsData[asset]["symbol"]

    # for each metric in the asset
    for metKey in metricsKeys:
        # if metric is in the list of metrics to use
        if metKey in useMetricKeys:
            # for each column within the metric key
            for k in metricsKeys[metKey]:
                # if column itself has a list of values 
                if type(assetsData[asset]["metrics"][metKey][k]) == dict:
                    # separate the list of values into separate columns
                    for dk in assetsData[asset]["metrics"][metKey][k]:
                        assetdict[k+"__"+dk] = assetsData[asset]["metrics"][metKey][k][dk]
                # otherwise store as column value
                else:
                    assetdict[k] = assetsData[asset]["metrics"][metKey][k]
        else:
            pass

    # for each profile in the asset
    for profKey in profileKeys:
        # if profile is in the list of profiles to use
        if profKey in useProfKeys:
            # for each column within the profile key
            for k in profileKeys[profKey]:
                if k in ["grants"]:
                    pass
                else:
                    # if column itself has a list of values 
                    if type(assetsData[asset]["profile"][profKey][k]) == dict:
                        # separate the list of values into separate columns
                        for dk in assetsData[asset]["profile"][profKey][k]:
                            # if profile:economics:block_explorer don't bother
                            if dk in ["block_explorers", "accounts", "multitoken"]:
                                pass
                            else:
                                #if recursive dictionary do it again
                                if type(assetsData[asset]["profile"][profKey][k][dk]) == dict:
                                    for ddk in assetsData[asset]["profile"][profKey][k][dk]:
                                        assetdict[dk+"__"+ddk] = assetsData[asset]["profile"][profKey][k][dk][ddk]
                                elif type(assetsData[asset]["profile"][profKey][k][dk]) == list:
                                    if len(assetsData[asset]["profile"][profKey][k][dk]) == 0:
                                        assetdict[k+"__"+dk] = None
                                    else:
                                        print(k+"__"+dk, assetsData[asset]["profile"][profKey][k][dk])
                                else:
                                    assetdict[k+"__"+dk] = assetsData[asset]["profile"][profKey][k][dk]

                    # otherwise store as column value
                    elif type(assetsData[asset]["profile"][profKey][k]) == list:
                        if len(assetsData[asset]["profile"][profKey][k]) == 0:
                            assetdict[k] = None
                        else:
                            pass
                    else:
                        assetdict[k] = assetsData[asset]["profile"][profKey][k]
        else:
            pass
    
    # append in master list
    masterlist.append(assetdict)

# store as dataframe
assetdf = pd.DataFrame(masterlist)
assetdf

,name,symbol,price_usd,price_btc,price_eth,volume_last_24_hours,real_volume_last_24_hours,volume_last_24_hours_overstatement_multiple,percent_change_usd_last_1_hour,percent_change_btc_last_1_hour,...,consensus__mining_algorithm,consensus__next_halving_date,consensus__is_victim_of_51_percent_attack,native_treasury__treasury_usage_details,governance_details,onchain_governance__onchain_governance_type,onchain_governance__onchain_governance_details,onchain_governance__is_treasury_decentralized,ohlcv_last_1_hour,ohlcv_last_24_hour
0,Bitcoin,BTC,34659.297687,1.000000e+00,13.532620,5.425426e+09,3.156681e+09,1.718712,-0.337203,0.000000,...,SHA-256,2024-05-10T04:00:00Z,False,None,Model Overview\nBitcoin governance is the proc...,None,None,False,NaN,NaN
1,Ethereum,ETH,2561.166777,7.389552e-02,1.000000,3.789751e+09,2.286683e+09,1.657314,-0.329556,0.007674,...,Ethash,None,False,None,Ethereum operates similarly to Bitcoin through...,No On-Chain Governance,While Ethereum does not have an official on-ch...,False,NaN,NaN
2,Tether,USDT,0.999732,2.884455e-05,0.000390,2.759426e+08,7.351523e+07,3.753543,-0.006887,0.331434,...,None,None,None,None,Tether Limited is the sole entity capable of a...,None,None,None,NaN,NaN
3,BNB,BNB,356.347521,1.028087e-02,0.139130,2.869418e+08,2.272412e+08,1.262719,-0.232414,0.099587,...,n/a,None,False,None,"Currently, BNB is governed in a centralized ma...",Upcoming,Parameters for the governance module are outli...,False,NaN,NaN
4,USD Coin,USDC,0.999903,2.884789e-05,0.000390,1.427853e+09,1.387032e+09,1.029430,-0.004791,0.327978,...,None,None,None,None,CENTRE members are the sole entities capable o...,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Anoncoin,ANC,1.593186,4.596381e-05,0.000622,1.766444e+07,1.317219e+07,1.341040,-0.996895,-0.668940,...,None,None,None,None,None,None,None,None,NaN,NaN
496,Lympo,LYM,0.004009,1.157563e-07,0.000002,2.916490e+05,4.428981e+02,658.501357,NaN,NaN,...,n/a,None,None,None,None,None,None,None,NaN,NaN
497,Unslashed Finance,USF,0.156222,4.395348e-06,0.000059,0.000000e+00,0.000000e+00,1.000000,NaN,NaN,...,None,None,None,None,None,None,None,None,NaN,NaN
498,Monavale,MONA,323.254834,8.986001e-03,0.119962,1.609967e+04,1.609967e+04,1.000000,NaN,NaN,...,None,None,None,None,None,None,None,None,NaN,NaN


In [12]:
# write to csv as well as plaintext
assetdf.to_csv("assets.csv", index=False)
with open('table_assets.txt', 'w') as f:
    f.write(tabulate(assetdf, headers='keys', tablefmt='grid'))